In [ ]:
import pandas as pd
import lecilab_behavior_analysis.utils as utils
from pathlib import Path
%load_ext autoreload
%autoreload 2


In [ ]:
# mouse = "mouse2"
# df = utils.load_example_data(mouse)

In [ ]:
import socket
socket.gethostname()

In [ ]:
# load data from cluster
tv_projects = utils.get_server_projects()
print(tv_projects)


In [ ]:

# see the available animals
animals = utils.get_animals_in_project(tv_projects[1])
print(animals)

In [ ]:
# download the data for a specific animal
mouse = "ACV007"
local_path = Path(utils.get_outpath()) / Path(tv_projects[1]) / Path("sessions") / Path(mouse)
# create the directory if it doesn't exist
local_path.mkdir(parents=True, exist_ok=True)
# download the session data
utils.rsync_session_data(
    project_name=tv_projects[1],
    animal=mouse,
    local_path=str(local_path),
    credentials=utils.get_idibaps_cluster_credentials(),
)

In [ ]:
# load the data
df = pd.read_csv(local_path / Path(f'{mouse}.csv'), sep=";")

In [ ]:
import lecilab_behavior_analysis.plots as plots
import lecilab_behavior_analysis.df_transforms as dft
df = dft.fill_missing_data(df)

# add a column with the date for the day
df = dft.add_day_column_to_df(df)

# create a figure with 1 axis for the calendar plot
import matplotlib.pyplot as plt
fig, ax_cal = plt.subplots(figsize=(10, 5), dpi=300)
# generate the calendar plot
dates_df = dft.get_dates_df(df)
cal_image = plots.rasterize_plot(plots.training_calendar_plot(dates_df), dpi=300)
# paste the calendar plot filling the entire axis
ax_cal.imshow(cal_image)
ax_cal.axis("off")

plt.show()

In [ ]:
from lecilab_behavior_analysis.figure_maker import subject_progress_figure
fig = subject_progress_figure(df, perf_window=100, summary_matrix_plot=False)

In [ ]:
from lecilab_behavior_analysis.figure_maker import session_summary_figure
from lecilab_behavior_analysis.df_transforms import add_trial_of_day_column_to_df, add_day_column_to_df
# select the session you want to plot
date = "2025-05-06"
df = add_day_column_to_df(df)
df = add_trial_of_day_column_to_df(df)
sdf = df[df["year_month_day"] == date]
fig = session_summary_figure(sdf, mouse, perf_window=15, width=10, height=5)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy
from scipy.optimize import curve_fit
import df_transforms as dft
from sklearn.linear_model import LogisticRegression
import seaborn as sns

In [ ]:
df_test = df.dropna(subset = ['visual_stimulus'])
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))
for i, ax in zip(df_test.groupby('current_training_stage').groups.keys(), [ax1, ax2]):
    df_test_sub = df_test[df_test["current_training_stage"] == i]
    df_test_sub['visual_stimulus_devi'] = df_test_sub['visual_stimulus'].apply(lambda x: abs(round(eval(x)[0] / eval(x)[1], 4)))
    # This was good in order to make the fit work for both left and right choices!
    df_test_sub['visual_stimulus_devi'] = df_test_sub.apply(
        lambda row: row['visual_stimulus_devi'] if row['correct_side'] == 'left' else -row['visual_stimulus_devi'],
        axis=1
    )
    df_test_sub = dft.add_mouse_first_choice(df_test_sub)
    df_test_sub['left_choice_new'] = df_test_sub['first_choice'].apply(lambda x: 1 if x == 'left' else 0)
    
    # Now we can fit the data and visualize the results
    X = df_test_sub['visual_stimulus_devi'].values.reshape(-1, 1)
    y = df_test_sub['left_choice_new'].values.astype(int)
    model = LogisticRegression()
    model.fit(X, y)

    # Now we have a model that predicts the probability of a left choice based on ANY visual stimulus deviation (xs).
    # For plotting, we can generate a range of values for the visual stimulus deviation
    import numpy as np
    xs = np.linspace(df_test_sub['visual_stimulus_devi'].min(), df_test_sub['visual_stimulus_devi'].max(), 100).reshape(-1, 1)
    y_prob = model.predict_proba(xs)[:, 1]

    # Plot the actual choices of the mouse

    sns.pointplot(
        x='visual_stimulus_devi',
        y='left_choice_new',
        data=df_test_sub,
        estimator=lambda x: np.mean(x),
        color='blue',
        markers='o',
        errorbar=("ci", 95),
        ax=ax,
        label='Observed Choices',
        native_scale= True,
        linestyles='',
    )

    # overlay the fitted logistic regression curve
    ax.plot(xs, y_prob, color='red', label='Logistic Regression Fit')
    ax.set_xlabel("Visual Stimulus Deviation")
    ax.set_ylabel("Probability of Left Choice")
    plt.title("Psychometric Curve")
    plt.legend()
plt.show()

In [ ]:
df_test = df.dropna(subset = ['visual_stimulus'])
df_test['visual_stimulus_devi'] = df_test['visual_stimulus'].apply(lambda x: abs(round(eval(x)[0] / eval(x)[1], 4)))
# This was good in order to make the fit work for both left and right choices!
df_test['visual_stimulus_devi'] = df_test.apply(
    lambda row: row['visual_stimulus_devi'] if row['correct_side'] == 'left' else -row['visual_stimulus_devi'],
    axis=1
)
df_test = dft.add_mouse_first_choice(df_test)
df_test['left_choice_new'] = df_test['first_choice'].apply(lambda x: 1 if x == 'left' else 0)

# By the way I am naming columns weirdly, just so you can play around with the different solutions and see how they work.
# Once we have what we need, we should clean up the code and use more meaningful names.

In [ ]:
df_test_sorted = df_test.sort_values(by='visual_stimulus_diff')
X = df_test_sorted['visual_stimulus_diff'].values.reshape(-1, 1)
y = df_test_sorted['left_choice'].values.astype(int)
model = LogisticRegression()
model.fit(X, y)
y_pred = model.predict(X)
y_prob = model.predict_proba(X)[:, 1]
plt.plot(X, y_prob)

plt.xlabel("Visual Stimulus Difference")
plt.ylabel("Probability of Left Choice")

In [ ]:
df_test = df_test[df_test["current_training_stage"] == "TwoAFC_visual_hard"]

In [ ]:
# Now we can fit the data and visualize the results
X = df_test['visual_stimulus_devi'].values.reshape(-1, 1)
y = df_test['left_choice_new'].values.astype(int)
model = LogisticRegression()
model.fit(X, y)

# Now we have a model that predicts the probability of a left choice based on ANY visual stimulus deviation (xs).
# For plotting, we can generate a range of values for the visual stimulus deviation
import numpy as np
xs = np.linspace(df_test['visual_stimulus_devi'].min(), df_test['visual_stimulus_devi'].max(), 100).reshape(-1, 1)
y_prob = model.predict_proba(xs)[:, 1]

# Plot the actual choices of the mouse
fig, ax = plt.subplots(figsize=(5, 5))
sns.pointplot(
    x='visual_stimulus_devi',
    y='left_choice_new',
    data=df_test,
    estimator=lambda x: np.mean(x),
    color='blue',
    markers='o',
    errorbar=("ci", 95),
    ax=ax,
    label='Observed Choices',
    native_scale= True,
    linestyles='',
)

# overlay the fitted logistic regression curve
ax.plot(xs, y_prob, color='red', label='Logistic Regression Fit')
ax.set_xlabel("Visual Stimulus Deviation")
ax.set_ylabel("Probability of Left Choice")
plt.title("Psychometric Curve")
plt.legend()
plt.show()

In [ ]:
df_test['visual_stimulus_diff'] = df_test['visual_stimulus'].apply(lambda x: abs(eval(x)[0] - eval(x)[1]))
df_test['visual_stimulus_diff'] = df_test.apply(
    lambda row: row['visual_stimulus_diff'] if row['correct_side'] == 'left' else -row['visual_stimulus_diff'],
    axis=1
)

In [ ]:
df_test['left_choice_new'] = df_test['first_choice'].apply(lambda x: 1 if x == 'left' else 0)

# Now we can fit the data and visualize the results
X = df_test.sort_values(by='visual_stimulus_diff')['visual_stimulus_diff'].values.reshape(-1, 1)
y = df_test.sort_values(by='visual_stimulus_diff')['left_choice_new'].values.astype(int)
model = LogisticRegression()
model.fit(X, y)
y_prob = model.predict_proba(X)[:, 1]

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

# Generate a grid of values for visualization
devi_range = np.linspace(df_test['visual_stimulus_devi'].min(), df_test['visual_stimulus_devi'].max(), 50)
diff_range = np.linspace(df_test['visual_stimulus_diff'].min(), df_test['visual_stimulus_diff'].max(), 50)
devi_grid, diff_grid = np.meshgrid(devi_range, diff_range)
interaction_grid = devi_grid * diff_grid

# Flatten the grid for prediction
grid_data = np.column_stack((devi_grid.ravel(), diff_grid.ravel(), interaction_grid.ravel()))
grid_data_const = sm.add_constant(grid_data)

# Predict probabilities using the model
probabilities = logit_model_multi.predict(grid_data_const).reshape(devi_grid.shape)

# Plot the 3D surface
fig = plt.figure(figsize=(10, 7))
ax = fig.add_subplot(111, projection='3d')
surf = ax.plot_surface(devi_grid, diff_grid, probabilities, cmap='viridis', alpha=0.8)
ax.set_xlabel('Visual Stimulus Deviation')
ax.set_ylabel('Visual Stimulus Difference')
ax.set_zlabel('Probability of Left Choice')
plt.title('Influence of Regressors on Probability of Left Choice')
fig.colorbar(surf, shrink=0.5, aspect=10)
plt.show()

In [ ]:
# It is interesting to compare the effects of the relative difference between the two visual stimuli,
# and the absolute difference between them.

# Maybe what we can do is to train another logistic regression model, adding as well the absolute difference
# between the two visual stimuli, and see how it affects the probability of a left choice.
# Do you know what I mean?

for i in df_test.groupby('visual_stimulus_devi'):
    df_i = i[1].sort_values(by='visual_stimulus_diff')
    X = df_i['visual_stimulus_diff'].values.reshape(-1, 1)
    y = df_i['left_choice'].values.astype(int)
    model = LogisticRegression()
    model.fit(X, y)
    y_pred = model.predict(X)
    y_prob = model.predict_proba(X)[:, 1]
    plt.plot(X, y_prob, label=f"Visual Stimulus Deviation: {i[0]}")
    plt.legend()
plt.xlabel("Visual Stimulus Difference")
plt.ylabel("Probability of Left Choice")

In [ ]:
# let's plot it as matrix, with the values being the probability of a left choice.
# in the x axis we will have the visual stimulus deviation,
# and in the y axis we will have the visual stimulus difference.
import seaborn as sns
# create a pivot table with the visual stimulus deviation and difference
pivot_table = df_test.pivot_table(
    index='visual_stimulus_diff_binned',
    columns='visual_stimulus_devi',
    values='left_choice_new',
    aggfunc='mean'
)
# plot the heatmap
plt.figure(figsize=(5, 5))
sns.heatmap(pivot_table, cmap='coolwarm', annot=True, fmt=".2f", cbar_kws={'label': 'Probability of Left Choice'})
plt.xlabel("Visual Stimulus Deviation")
plt.ylabel("Visual Stimulus Difference")
plt.title("Heatmap of Probability of Left Choice")
import statsmodels.api as sm

# Add an interaction term between visual_stimulus_devi and visual_stimulus_diff
def interaction_calc(row):
    is_left = 1 if row['correct_side'] == 'left' else -1
    return row['visual_stimulus_devi'] * row['visual_stimulus_diff_binned'] * is_left
df_test['interaction_term'] = df_test.apply(interaction_calc, axis=1)


# Prepare the independent variables
X_multi = df_test[['visual_stimulus_devi', 'visual_stimulus_diff_binned', 'interaction_term']]
X_multi_const = sm.add_constant(X_multi)
y = df_test['left_choice_new'].values.astype(int)

# Fit the logistic regression model with multiple regressors
logit_model_multi = sm.Logit(y, X_multi_const).fit()

# Display the summary, which includes p-values for all regressors
print(logit_model_multi.summary())

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

# Generate a grid of values for visualization
devi_range = np.linspace(df_test['visual_stimulus_devi'].min(), df_test['visual_stimulus_devi'].max(), 50)
diff_range = np.linspace(df_test['visual_stimulus_diff'].min(), df_test['visual_stimulus_diff'].max(), 50)
devi_grid, diff_grid = np.meshgrid(devi_range, diff_range)
interaction_grid = devi_grid * diff_grid

# Flatten the grid for prediction
grid_data = np.column_stack((devi_grid.ravel(), diff_grid.ravel(), interaction_grid.ravel()))
grid_data_const = sm.add_constant(grid_data)

# Predict probabilities using the model
probabilities = logit_model_multi.predict(grid_data_const).reshape(devi_grid.shape)

# Plot the 3D surface
fig = plt.figure(figsize=(10, 7))
ax = fig.add_subplot(111, projection='3d')
surf = ax.plot_surface(devi_grid, diff_grid, probabilities, cmap='viridis', alpha=0.8)
ax.set_xlabel('Visual Stimulus Deviation')
ax.set_ylabel('Visual Stimulus Difference')
ax.set_zlabel('Probability of Left Choice')
plt.title('Influence of Regressors on Probability of Left Choice')
fig.colorbar(surf, shrink=0.5, aspect=10)
plt.show()

In [ ]:
# Now, instead of the difference, let's use the absolute value of the lowest visual stimulus
# This is the absolute value of the lowest visual stimulus
df_test['visual_stimulus_lowest'] = df_test['visual_stimulus'].apply(lambda x: abs(eval(x)[0]) if eval(x)[0] < eval(x)[1] else abs(eval(x)[1]))
# create 10 bins for the absolute value of the lowest visual stimulus
min_value = df_test['visual_stimulus_lowest'].min()
max_value = df_test['visual_stimulus_lowest'].max()
bins = np.linspace(min_value, max_value, 11)
df_test['visual_stimulus_lowest_binned'] = pd.cut(df_test['visual_stimulus_lowest'], bins=bins, labels=[f"{b:.2f}" for b in bins[:-1]])
# create a pivot table with the visual stimulus deviation and absolute value of the lowest visual stimulus
pivot_table_abs = df_test.pivot_table(
    index='visual_stimulus_lowest_binned',
    columns='visual_stimulus_devi',
    values='left_choice_new',
    aggfunc='mean',
    observed=True
)
# plot the heatmap
plt.figure(figsize=(5, 5))
sns.heatmap(pivot_table_abs, cmap='coolwarm', annot=True, fmt=".2f", cbar_kws={'label': 'Probability of Left Choice'})
plt.xlabel("Visual Stimulus Deviation")
plt.ylabel("Absolute Value of Lowest Visual Stimulus")
plt.title("Heatmap of Probability of Left Choice")
plt.show()

In [ ]:
sns.pointplot(
    x='visual_stimulus_diff_binned',
    y='left_choice_new',
    data=df_test,
    estimator=lambda x: np.mean(x),
    color='blue',
    markers='o',
    errorbar=("ci", 95),
    native_scale=True,
    linestyles='',
)

In [ ]:
# let's plot it as matrix, with the values being the probability of a left choice.
# in the x axis we will have the visual stimulus deviation,
# and in the y axis we will have the visual stimulus difference.
import seaborn as sns
# create a pivot table with the visual stimulus deviation and difference
pivot_table = df_test.pivot_table(
    index='visual_stimulus_diff_binned',
    columns='visual_stimulus_devi',
    values='left_choice_new',
    aggfunc='mean'
)
# plot the heatmap
plt.figure(figsize=(5, 5))
sns.heatmap(pivot_table, cmap='coolwarm', annot=True, fmt=".2f", cbar_kws={'label': 'Probability of Left Choice'})
plt.xlabel("Visual Stimulus Deviation")
plt.ylabel("Visual Stimulus Difference")
plt.title("Heatmap of Probability of Left Choice")
plt.show()

In [ ]:
# Now, instead of the difference, let's use the absolute value of the lowest visual stimulus
# This is the absolute value of the lowest visual stimulus
df_test['visual_stimulus_lowest'] = df_test['visual_stimulus'].apply(lambda x: abs(eval(x)[0]) if eval(x)[0] < eval(x)[1] else abs(eval(x)[1]))
# create 10 bins for the absolute value of the lowest visual stimulus
min_value = df_test['visual_stimulus_lowest'].min()
max_value = df_test['visual_stimulus_lowest'].max()
bins = np.linspace(min_value, max_value, 11)
df_test['visual_stimulus_lowest_binned'] = pd.cut(df_test['visual_stimulus_lowest'], bins=bins, labels=[f"{b:.2f}" for b in bins[:-1]])
# create a pivot table with the visual stimulus deviation and absolute value of the lowest visual stimulus
pivot_table_abs = df_test.pivot_table(
    index='visual_stimulus_lowest_binned',
    columns='visual_stimulus_devi',
    values='left_choice_new',
    aggfunc='mean',
    observed=True
)
# plot the heatmap
plt.figure(figsize=(5, 5))
sns.heatmap(pivot_table_abs, cmap='coolwarm', annot=True, fmt=".2f", cbar_kws={'label': 'Probability of Left Choice'})
plt.xlabel("Visual Stimulus Deviation")
plt.ylabel("Absolute Value of Lowest Visual Stimulus")
plt.title("Heatmap of Probability of Left Choice")
plt.show()

Work on multiple animals

In [ ]:
df_dic = {}
for mouse in animals:
    local_path = Path(utils.get_outpath()) / Path(tv_projects[1]) / Path("sessions") / Path(mouse)
    # create the directory if it doesn't exist
    local_path.mkdir(parents=True, exist_ok=True)
    # download the session data
    utils.rsync_session_data(
        project_name=tv_projects[1],
        animal=mouse,
        local_path=str(local_path),
        credentials=utils.get_idibaps_cluster_credentials(),
    )
    # load the data
    df_dic[mouse] = pd.read_csv(local_path / Path(f'{mouse}.csv'), sep=";")

In [ ]:
df_dic_hard = {}
for df_name, df in zip(df_dic.keys(), df_dic.values()):
    if 'TwoAFC_visual_hard' in df["current_training_stage"].unique():
        df = df.dropna(subset = ['visual_stimulus'])
        df = df[df["current_training_stage"] == "TwoAFC_visual_hard"]

        df['visual_stimulus_devi'] = df['visual_stimulus'].apply(lambda x: abs(round(eval(x)[0] / eval(x)[1], 4)))
        df['visual_stimulus_devi'] = df.apply(
            lambda row: row['visual_stimulus_devi'] if row['correct_side'] == 'left' else -row['visual_stimulus_devi'],
            axis=1
        )
        df['visual_stimulus_diff'] = df['visual_stimulus'].apply(lambda x: abs(eval(x)[0] - eval(x)[1]))
        df['visual_stimulus_diff'] = df.apply(
            lambda row: row['visual_stimulus_diff'] if row['correct_side'] == 'left' else -row['visual_stimulus_diff'],
            axis=1
        )
        df["visual_stimulus_diff_binned"] = df['visual_stimulus_diff'] // 0.1
        df = dft.add_mouse_first_choice(df)
        df['left_choice_new'] = df['first_choice'].apply(lambda x: 1 if x == 'left' else 0)
        
        df_dic_hard[df_name] = df

In [ ]:
df_devi_diffBin_inter_p = pd.DataFrame()
df_devi_diffBin_inter_coef = pd.DataFrame()
for df_name, df in zip(df_dic_hard.keys(), df_dic_hard.values()):
    df['interaction_term'] = df.apply(interaction_calc, axis=1)
    # Prepare the independent variables
    X_multi = df[['visual_stimulus_devi', 'visual_stimulus_diff_binned', 'interaction_term']]
    X_multi_const = sm.add_constant(X_multi)
    y = df['left_choice_new'].values.astype(int)

    # Fit the logistic regression model with multiple regressors
    logit_model_multi = sm.Logit(y, X_multi_const).fit()

    df_devi_diffBin_inter_p[df_name] = logit_model_multi.pvalues
    df_devi_diffBin_inter_coef[df_name] = logit_model_multi.params

In [ ]:
df_devi_diff_inter_p = pd.DataFrame()
df_devi_diff_inter_coef = pd.DataFrame()
for df_name, df in zip(df_dic_hard.keys(), df_dic_hard.values()):
    df['interaction_term'] = df.apply(interaction_calc, axis=1)
    # Prepare the independent variables
    X_multi = df[['visual_stimulus_devi', 'visual_stimulus_diff', 'interaction_term']]
    X_multi_const = sm.add_constant(X_multi)
    y = df['left_choice_new'].values.astype(int)

    # Fit the logistic regression model with multiple regressors
    logit_model_multi = sm.Logit(y, X_multi_const).fit()

    df_devi_diff_inter_p[df_name] = logit_model_multi.pvalues
    df_devi_diff_inter_coef[df_name] = logit_model_multi.params

In [ ]:
df_devi_diffBin_inter_p.rename(index={'visual_stimulus_devi': 'devi', 'visual_stimulus_diff_binned': 'diff', 'interaction_term': 'inter'}, inplace=True)
df_devi_diff_inter_p.rename(index={'visual_stimulus_devi': 'devi', 'visual_stimulus_diff': 'diff', 'interaction_term': 'inter'}, inplace=True)
for df_name, color in zip(df_dic_hard.keys(), sns.color_palette("colorblind", len(df_dic_hard))):
    plt.plot (df_devi_diff_inter_p[df_name], label=df_name+ 'devi_diff_inter', color=color)
    plt.plot (df_devi_diffBin_inter_p[df_name], label=df_name+ 'devi_diffBin_inter', color=color, linestyle='--')
plt.axhline(y=0.05, color='k', linestyle='--', label='p-value threshold')
plt.xlabel("Regressors")
plt.ylabel("p-value")
plt.legend(loc = (1 , 0))

In [ ]:
df_devi_diffBin_inter_coef.rename(index={'visual_stimulus_devi': 'devi', 'visual_stimulus_diff_binned': 'diff', 'interaction_term': 'inter'}, inplace=True)
df_devi_diff_inter_coef.rename(index={'visual_stimulus_devi': 'devi', 'visual_stimulus_diff': 'diff', 'interaction_term': 'inter'}, inplace=True)
for df_name, color in zip(df_dic_hard.keys(), sns.color_palette("colorblind", len(df_dic_hard))):
    plt.plot (df_devi_diff_inter_coef[df_name], label=df_name+ 'devi_diff_inter', color=color)
    plt.plot (df_devi_diffBin_inter_coef[df_name], label=df_name+ 'devi_diffBin_inter', color=color, linestyle='--')
plt.xlabel("Regressors")
plt.ylabel("Coefficient")
plt.legend(loc = (1 , 0))